# ДЗ: Векторная БД, индексация и поиск (полное решение)

**Цель:** выполнить требования задания на одном рабочем ноутбуке: от индексации до сравнения качества/скорости.

**Что будет в ноутбуке:**
- выбор БД (Chroma) и её настройка
- подготовка датасета (AG News)
- эмбеддинги и индексация
- базовый и продвинутый поиск
- сравнение качества vs скорость
- (бонус) гибридный поиск и batch‑запросы

> Если у вас есть свой датасет, замените блок загрузки данных — остальная логика останется.

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  datasets \
  numpy \
  pydantic==2.12.3 \
  requests==2.32.4

## Если падает numpy/scipy

В Colab иногда ломается связка `numpy`/`scipy`. Зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U numpy==1.26.4 scipy==1.11.4

> Важно: после понижения `numpy/scipy` Colab может предупредить о конфликте с JAX и другими пакетами. Это **нормально**, если вы не используете эти пакеты в этом ноутбуке. Игнорируйте предупреждения.

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Часть 1. Настройка и индексация

### 1.1 Выбор БД
Мы используем **Chroma** — лёгкую векторную БД с HNSW (ANN) под капотом.

### 1.2 Подготовка датасета
Возьмём открытый датасет **AG News** (4 класса новостей).

In [ ]:
from datasets import load_dataset

LABELS = {0: "world", 1: "sports", 2: "business", 3: "sci_tech"}

# Берем 2000 документов (можно увеличить/уменьшить)
N_DOCS = 2000
raw_ds = load_dataset("ag_news", split=f"train[:{N_DOCS}]")

texts = [x["text"] for x in raw_ds]
labels = [LABELS[x["label"]] for x in raw_ds]

metadatas = [{"label": lbl, "source": "ag_news", "source_id": i} for i, lbl in enumerate(labels)]
ids = [f"doc-{i}" for i in range(len(texts))]

print("Документов:", len(texts))
print("Пример:", texts[0][:200])

### 1.3 Эмбеддинги и индексация

Мы создаём эмбеддинги и индексируем документы в Chroma.

**Основные параметры HNSW (ANN):**
- `hnsw:M` — количество связей в графе (больше = точнее, но тяжелее)
- `hnsw:construction_ef` — качество построения индекса (больше = точнее, но медленнее)
- `hnsw:search_ef` — качество поиска (больше = точнее, но медленнее)

Здесь задаём `hnsw:space="cosine"` и разумные параметры.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from pathlib import Path

store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="ag_news")

persist_dir = Path("./db/chroma_ag_news")

collection_metadata = {
    "hnsw:space": "cosine",
    "hnsw:M": 16,
    "hnsw:construction_ef": 200,
    "hnsw:search_ef": 64,
}

vectorstore = Chroma(
    collection_name="ag_news_cosine",
    embedding_function=cached_embedder,
    persist_directory=str(persist_dir),
    collection_metadata=collection_metadata,
)

# Добавляем документы (эмбеддинги считаются под капотом)
vectorstore.add_texts(texts=texts, metadatas=metadatas, ids=ids)

# Сохраняем на диск, если метод доступен
if hasattr(vectorstore, "persist"):
    vectorstore.persist()

print("Индекс готов")

## Часть 2. Реализация поиска

### 2.1 Базовый семантический поиск
Настраиваем `top-k` и проверяем результаты.

In [ ]:
query = "New regulations for banks and market growth"
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results, 1):
    print(f"\nРезультат {i} (label={doc.metadata.get('label')}):")
    print(doc.page_content[:200])

### 2.2 Фильтрация по метаданным

Фильтруем результаты, например только по теме `business`.

In [ ]:
filtered = vectorstore.similarity_search(
    "Company profits and market share",
    k=3,
    filter={"label": "business"},
)

for i, doc in enumerate(filtered, 1):
    print(f"\nФильтрованный результат {i} (label={doc.metadata.get('label')}):")
    print(doc.page_content[:200])

### 2.3 Настройка similarity metrics

Chroma поддерживает разные пространства:
- `cosine`
- `l2`
- `ip` (inner product)

Ниже создадим вторую коллекцию на `l2` и сравним.

In [ ]:
vectorstore_l2 = Chroma(
    collection_name="ag_news_l2",
    embedding_function=cached_embedder,
    persist_directory=str(persist_dir),
    collection_metadata={
        "hnsw:space": "l2",
        "hnsw:M": 16,
        "hnsw:construction_ef": 200,
        "hnsw:search_ef": 64,
    },
)

# Заполняем второй индекс (можно пропустить, если уже есть)
vectorstore_l2.add_texts(texts=texts, metadatas=metadatas, ids=[f"l2-{i}" for i in range(len(texts))])

res_cos = vectorstore.similarity_search("Stock market and company earnings", k=3)
res_l2 = vectorstore_l2.similarity_search("Stock market and company earnings", k=3)

print("COSINE:")
for d in res_cos:
    print("-", d.metadata.get("label"))

print("L2:")
for d in res_l2:
    print("-", d.metadata.get("label"))

### 2.4 Trade‑offs: скорость vs точность

Мы сравним точный поиск (brute‑force) и ANN (HNSW) по `recall@k` и времени.

In [ ]:
import numpy as np
import time

# Эмбеддинги для документов (для честного brute‑force сравнения)
# ВНИМАНИЕ: это может занять время и потратить лимит API
emb_matrix = np.array(embeddings.embed_documents(texts))

# Берем несколько запросов
queries = [
    "International conflict and diplomacy",
    "Football match and team victory",
    "Stock market and company profits",
    "New AI technology and research",
]

k = 5

# Точный поиск: cosine similarity
emb_matrix_norm = emb_matrix / np.linalg.norm(emb_matrix, axis=1, keepdims=True)

exact_ids = []
start = time.perf_counter()
for q in queries:
    q_emb = np.array(embeddings.embed_query(q))
    q_emb = q_emb / np.linalg.norm(q_emb)
    sims = emb_matrix_norm @ q_emb
    topk = np.argsort(-sims)[:k]
    exact_ids.append(set(topk.tolist()))
exact_time = time.perf_counter() - start

# ANN поиск (HNSW через Chroma)
ann_ids = []
start = time.perf_counter()
for q in queries:
    docs = vectorstore.similarity_search(q, k=k)
    ids_found = set(int(d.metadata.get("source_id")) for d in docs)
    ann_ids.append(ids_found)
ann_time = time.perf_counter() - start

# Recall@k
recalls = []
for e, a in zip(exact_ids, ann_ids):
    if not a:
        recalls.append(0.0)
    else:
        recalls.append(len(e & a) / len(e))

print("Exact time:", round(exact_time, 3), "s")
print("ANN time:", round(ann_time, 3), "s")
print("Recall@k:", round(sum(recalls) / len(recalls), 3))

### 2.5 Сравнение параметров ANN (search_ef)

Сравним скорость и recall при разных `hnsw:search_ef` (меньше = быстрее, больше = точнее).

> Чтобы не тратить много времени/лимита, используйте небольшой поднабор документов.

In [ ]:
def build_collection(name: str, search_ef: int):
    vs = Chroma(
        collection_name=name,
        embedding_function=cached_embedder,
        persist_directory=str(persist_dir),
        collection_metadata={
            "hnsw:space": "cosine",
            "hnsw:M": 16,
            "hnsw:construction_ef": 200,
            "hnsw:search_ef": search_ef,
        },
    )
    vs.add_texts(texts=texts, metadatas=metadatas, ids=[f"{name}-{i}" for i in range(len(texts))])
    return vs

fast_vs = build_collection("ag_news_fast", search_ef=32)
accurate_vs = build_collection("ag_news_acc", search_ef=128)


def eval_recall(vs, queries, exact_ids, k=5):
    ann_ids = []
    start = time.perf_counter()
    for q in queries:
        docs = vs.similarity_search(q, k=k)
        ann_ids.append(set(int(d.metadata.get("source_id")) for d in docs))
    elapsed = time.perf_counter() - start

    recalls = []
    for e, a in zip(exact_ids, ann_ids):
        recalls.append(len(e & a) / len(e))
    return elapsed, sum(recalls) / len(recalls)

fast_time, fast_recall = eval_recall(fast_vs, queries, exact_ids, k=k)
acc_time, acc_recall = eval_recall(accurate_vs, queries, exact_ids, k=k)

print("FAST  -> time:", round(fast_time, 3), "s | recall@k:", round(fast_recall, 3))
print("ACCUR -> time:", round(acc_time, 3), "s | recall@k:", round(acc_recall, 3))

## Бонус: Hybrid search (BM25 + вектор)

Сравним чистый векторный поиск с гибридным.

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document

# Готовим документы для BM25
bm_docs = [Document(page_content=t, metadata=m) for t, m in zip(texts, metadatas)]

bm25 = BM25Retriever.from_documents(bm_docs)
bm25.k = 3

ensemble = EnsembleRetriever(
    retrievers=[bm25, vectorstore.as_retriever(search_kwargs={"k": 3})],
    weights=[0.4, 0.6],
)

query = "international diplomacy and conflict"
res_ens = ensemble.get_relevant_documents(query)

print("Hybrid results:")
for d in res_ens:
    print("-", d.metadata.get("label"))

## Бонус: Batch processing для запросов

Батчим запросы для ускорения и снижения накладных расходов.

In [ ]:
batch_queries = [
    "stock market growth",
    "football championship match",
    "new smartphone release",
    "government policy changes",
]

for q in batch_queries:
    docs = vectorstore.similarity_search(q, k=2)
    print("\nQ:", q)
    print("Top labels:", [d.metadata.get("label") for d in docs])

### 2.4 Trade‑offs: скорость vs точность

Мы сравним точный поиск (brute‑force) и ANN (HNSW) по `recall@k` и времени.

### 2.2 Фильтрация по метаданным

Фильтруем результаты, например только по теме `business`.